# Fullstack Terraform Lab

### Introduction

In this lesson, we'll use terraform to automatically deploy a our flask application to AWS.  In doing this, we'll need to set up an RDS instance, as well as an EC2 instance.  We'll also need to automatically setup our EC2 instance to download the images for the flask backend, the streamlit frontend, and of course start up the containers.

Let's get started.

### Building our backend

If you look at our llm-scraper codebase, you'll see that we currently have folders for `api` and `frontend`.  These folders are for holding our frontend streamlit application and our backend flask application.

Let's start with our backend application.  Inside the api folder, we want it to look like the following.

```bash
Dockerfile
.env
.flaskenv
├── app
│   ├── __init__.py
│   ├── data
│   ├── models
│   ├── requirements.txt
│   ├── server.py
│   ├── settings.py
│   └── setup.py
```

* So notice that we moved the `.env`, `.flaskenv`, `server.py` files into the api folder, as everything here is specific to the api.
* We also changed variables like `dev_db` to `db_conn` in our `server.py` file, and changed `DEV_DB` to `DB_CONN` in the `.env` file.  This makes sense as we will not always be connecting to the development database.

Now, we cannot directly just build our codebase in a Docker image -- as there is some initial setup that we'll need to complete.  Namely, we'll want to add code that will allow our flask application to setup our database.

* Database setup

To do this we'll add two methods into the `server.py` file.

```python
@app.cli.command("init-db")
def init_db_command():
    """Create database tables and seed data."""
    db.create_all()

# seed-db command
```

Ok, so the first function adds a cli command called `init-db` that will create all of the tables -- derived from our sqlalchemy models.  

> Give it a shot by setting the environmental variables to connect to a local database.  (You can just create a new database, and replace the `db_conn` variable with the connection to the database.  For example, if you connect to postgres, and create a database called `sample_scraper`.

You can comment out the original db_conn, and update `db_conn` to be:

`db_conn = 'postgresql://localhost/sample_scraper'`

Then from the folder that has `server.py` defined, run `flask init-db`, and the connect to the `sample_scraper` database to confirm that both the positions and scrapings tables have been created.

> <img src="./sample_tables.png" width="40%">

Next will be your turn to add a command line function.  In the `server.py` file, 

The command should be `seed-db`, which should decorate a function called `seed_db` which does the following:

* Counts the number of scrapings
* Counts the number of positions
* prints the number of scrapings and positions with some text like, "`Will seed scrapings and positions if there are none in the db.  Currently there are ... scrapings and ... positions`"
* Then only seed scrapings if there are zero in the database, and only seed positions if there are zero in the database.
* use the `seed_scrapings_from_csv` and `seed_positions_from_csv` functions, which are already defined in the `setup.py` file.

Test out your function by calling `seed-db` from the command line and confirm that there are scrapings and positions in the database.

Also, call `flask run`, and visit `localhost:5000/positions` to confirm that our flask api is serving our seeded positions.

* **Reset the db_conn:** Ok, so now we'll want to go back to the `settings.py` file, and make sure we are back to using our original `db_conn` string.

```python
db_conn = f'postgresql://{username}:{password}@{host}/{database}'
```

This is because we want to make sure that our db_conn string references the environmental variables, as docker will let us to pass environmental variables when we boot up our container.

> Note: Even if we have environmental variables in the `.env` file, any environmental variables we specify with the `docker run -e ` command will overwrite those in the `.env` file.  This is a good thing -- it allows us to change those variables more easily.

* Set up an AWS database

At this point, it's probably good to set up an rds instance, and record the variables of `username`, `password`, `host`, and `database`.  You can place them in the `.env` file if you like, or pass them when booting up the container (ie. at runtime). 

### Setting up docker

Ok, so now let's build the docker image.  The Dockerfile is a little tricky, so we have done this for you.  The key issue is that we want multiple things to occur when we boot up our docker container (aka "at runtime").  When we create a docker container we want to:

* Create our database tables (if they do not already exist)
* Seed our `positions` and `scrapings` tables if they do not already have data in them.
* Run our flask application by default.  

Ok, so to achieve this we do a couple of things:

1. Using entrypoint and command in our Dockerfile

If you look at the Dockerfile, you'll see the following towards the bottom.

```Dockerfile
ENTRYPOINT ["sh", "./setup.sh"]

CMD ["flask", "run", "--host=0.0.0.0"]
```

The `Entrypoint` is always run at run time, and CMD are the default arguments passed to what's specified in entrypoint.  So in this case, it's as if we are doing:

`sh ./setup.sh flask run --host=0.0.0.0`

So this will run the `./setup.sh` and then pass `flask run --host=0.0.0.0` to that file.

What will the `setup.sh` file do with the `flask run --host=0.0.0.0` argument?

2. `setup.sh` file

If you look at the setup.sh file you'll see the following:

```bash
flask init-db
flask seed-db

exec "$@"
```

So this will call our `init-db` and `seed-db` functions to create and seed our tables.  The `exec "$@"` allows us to pass optional bash commands to the script.  So when we setup our Dockerfile to run the script with:

`sh ./setup.sh flask run --host=0.0.0.0`

The arguments of `flask run --host=0.0.0.0` will be run in that last line.

We can play around with this.  For example, if we run `sh setup.sh echo hello world`, then we will have created and seeded our tables and run displayed hello world at the end.

So in this scenario, the `CMD ["flask", "run", "--host=0.0.0.0"]` says to pass `flask run --host=0.0.0.0` to our entrypoint `sh setup.sh`, and then the `setup.sh` file executes the `flask run` command after first creating the tables and seeding the database.

And remember we can override that default command at run time with something like:

`docker run image_name flask run --debug=True`

And that means that the setup.sh script will catch those arguments and run that instead of the original command.

* Building our image

Ok, so back to the show.  Now build the image, but do so with tagging the image with your dockerhub usernamem first.  Here's an example, so swap our `jek2141` with your username.

```bash
docker build -t jek2141/scraper_backend .
```

So now we'll want to bootup our image locally before trying it on our ec2 instance, but doing so will be a fairly long line.  So you may want to write it out in the `ec2-setup.sh` file, and the copy and paste it into your terminal.  

Ok, so boot up your container, but make sure you pass through environmental variables `docker build -e` for all of the database environmental variables. 

If it works, you should be able to go to `localhost:5000/positions` and see the positions in the flask application.  

* Make sure that your environmental variables are properly getting passed through by passing through some incorrect information (like a wrong password) that should cause your application to break.  If this doesn't work it means you are likely reading from the .env file but not from your `docker build -e` arguments.

Once you have a docker run command that is properly working, copy it into your `ec2-setup.sh` file, as you'll want this (or something like it) later on.

* One more thing

Now we're about to move onto terraform, but there is one issue with our docker image that we'll likely run into.  It's that there may be a mismatch between our laptop where we built our image on and the ec2 machine we ultimately use.  So before moving on, let's rebuild the image and tag it.  

> Just replace `jek2141` with your username.

`docker build -t jek2141/scraper_backend:amd_v2 --platform=linux/amd64/v2 .`

And now this is the image we'll ultimate want to use on our ec2 machine, so let's push it up to dockerhub.

`docker push jek2141/scraper_backend:amd_v2`

### Moving to Terraform

Ok, so now we're ready to try building our terraform infrastructure.  

A. Setting up the database

Ok, so we did a good amount of the work on the rds instance.  You can see it setup in the `tf/rds.tf` file.  Here's what you'll still need to do:
    
1. Use local variables
    * So currently `db_name`, `username`, `password` and `db_name` are hardcoded.  Move to using the local variables that are defined above.
    * In production you can look into using the aws secrets manager, or you could imagine just having the local variables in a `tf` file that is in a .gitignore and so not pushed up to github.  But you can skip that.

2. Attach the security group 

* We already declared a security group for you, and notice that we also set `publicly_accessible = true`.  Still, you'll need to *attach* the security group to the rds instance.

You can do this by adding a property of:

* `vpc_security_group_ids` and associating the security group id from the security group defined in the file.





> Later will need to add the : 
    
`security_groups = [aws_security_group.web_app.id]`

`docker build -t jek2141/scraper_backend --platform=linux/amd64/v2 .`

* `flask run`
* Add command lines arguments in the 

2. Switch to a production database

Fill in the .env file

3. Get working with docker

* We encoded this in a entrypoint.sh file
* `cd ./api`
* `docker build -t scraper_backend .`
* Notice the difference with entrypoint vs cmd functions

Notice that we can override any of the environmental variables with the -e flag

* docker run -p 5000:5000 -e DB_PASSWORD=foobar scraper_backend

* Connect the streamlit frontend app

* Notice in the docker file

* in the index 
* `st.write('hello all')`
* `CMD ["streamlit", "run", "./index.py", "--server.port=8501", "--server.address=0.0.0.0"]`

* `docker run -p 8503:8501 scraper_frontend`

Then connect frontend to backend.

* `docker network create my-network`

* `docker run -p 5000:5000 --network my-network --name backend scraper_backend`

* `docker run -p 8503:8501 --network my-network scraper_frontend`

* Create a `ec2-setup.sh` file

Now that we understand how if else and echo works, update the shell script so that it only will stop and remove the container if the backend container already exists.  And applies a separate if block for the frontend container.

* Update by pushing to dockerhub, and then update script to pull

* Update the docker run backend script to pass through env variables -- declare them as variables up top, and then reference them in the backend script.

* Confirm that connects to the rds database with DB_HOST
* Then go to localhost to confirm frontend works
* And exec into the backend machine, and make sure the DB_HOST environmental variable is properly set

> <img src="./db-host-env.png">

### Moving to Terraform

1. Create the database

* Confirm that you can access it by using `psql` to access from the command line.  Confirm that there is a database called `job_scraper`.

> <img src="./job-scraper-access.png">

2. Creates the ec2 instance and make sure we have database access

To achieve this, on the database's security group, we should add a `security_groups` tag of the ec2 instance's security group.

```bash
resource "aws_security_group" "postgres_access" {
  name = "scraper psql access"
  ingress {
    from_port   = 5432
    to_port     = 5432
    protocol    = "tcp"
    cidr_blocks = ["0.0.0.0/0"]
    security_groups = [aws_security_group.web_app.id]
  }
}
```

For example, above, notice that we not only give access on any port with the `0.0.0.0` but also give access to those resources that have a security group of `aws_security_group.web_app.id`.

* Confirm we can access from our ec2 machine

* sudo apt-get update

<img src="./psql-access.png" width="50%">

* sudo apt-get install postgresql-client-common
* sudo apt-get install postgresql-client

Then let's make sure the rest of our setup steps work on our ec2 machine.  As each is successful, add it to the `ec2-setup.sh` file.

* `sudo apt-get install docker -Y`

This looked like it worked, but if we run, `docker --version` to confirm it's installed, we see that it isn't available.  Let's try a different way of installing docker.

`sudo snap install docker`

Then confirm that this worked, by again running `docker --version`.  Ok, if this was successful, add it to the `ec2-setup.sh` file.

And `sudo docker ps` shows that it is up and running.

* So now let's scp our script over to the ec2 machine, and confirm that it works on the ec2 machine.
    > Before doing so, make sure the `DB_HOST` variable and `DB_NAME` point to the correct names

```bash
scp -i ~/.ssh/example.pem ./ec2-setup.sh  ubuntu@ec2-3-86-210-55.compute-1.amazonaws.com:/home/ubuntu/ec2-setup.sh
# ec2-setup.sh
```


* `sudo sh ec2-setup.sh`

Unfortunately fails on the following lines

* `docker run -d -p 5000:5000 --network $NETWORK_NAME --name $BACKEND_CONTAINER -e DB_USERNAME=$DB_USERNAME -e DB_PASSWORD=$DB_PASSWORD -e DB_NAME=$DB_NAME -e DB_HOST=$DB_HOST $BACKEND_IMAGE`

And this is because there is a mismatch where the software is `linux/amd64/v2`

Ideally we would specify to rebuild with 
* `docker build -t jek2141/scraper_backend:amd_v2 --platform=linux/amd64/v2 .`
* `docker build -t jek2141/scraper_frontend:amd_v2 --platform=linux/amd64/v2 .`

* Confirm it works with:
    
`curl localhost:5000/positions`

* Then move to a template file (and add the `depends on`)

* Check the console
    * terraform console data.rendered 
    
* Make sure to replace any call to docker with `sudo docker`



```bash
cat /var/log/cloud-init-output.log
cat /var/log/cloud-init.log
```

* Expose ports to the api, and ports to the 

* removed
```
while ! nc -z $DB_HOST 5432; do
    echo 'Waiting for DB to be ready'
    sleep 5
done
```

<img src="./complete-deployment.png" width="60%">

### Resources

[Terraform working with Following](https://discuss.hashicorp.com/t/template-v2-2-0-does-not-have-a-package-available-mac-m1/35099/3)